In [ ]:
import pandas as pd
dataset = pd.read_csv('/home/nesma/SemesterII/Neural Networks/Project/multilingual-text-categorization-dataset/data/dataset.csv', sep='\t', header=None).applymap(str)
dataset.columns = ["Language","label","text"]
languagesData=[]
loc = 0
languages = dataset[dataset.columns[0]].unique()
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1
    
englishData = englishData[["label","text"]]

englishData['label'] = '__label__' + englishData['label'].astype(str)

    
englishData.iloc[0:int(len(englishData)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.8):int(len(englishData)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [ ]:
englishData.head()

In [2]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-04-28 22:28:56,646 Reading data from .
2019-04-28 22:28:56,647 Train: train.csv
2019-04-28 22:28:56,648 Dev: dev.csv
2019-04-28 22:28:56,648 Test: test.csv


In [3]:
word_embeddings = [FlairEmbeddings('news-forward-fast')]

In [4]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=True)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=10)

2019-04-28 22:29:47,376 ----------------------------------------------------------------------------------------------------
2019-04-28 22:29:47,377 Evaluation method: MICRO_F1_SCORE
2019-04-28 22:29:47,379 ----------------------------------------------------------------------------------------------------


/home/nesma/anaconda3/envs/ImageProcessingLab/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


2019-04-28 22:32:23,000 epoch 1 - iter 0/113 - loss 0.02175651


RuntimeError: $ Torch: not enough memory: you tried to allocate 7GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1550780889552/work/aten/src/TH/THGeneral.cpp:201

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load_from_file('./best-model.pt')
sentence = Sentence('This is the best product that you can see ever, click here to buy it')
classifier.predict(sentence)
print(sentence.labels)